# Tuberculosis Diagnosis using VGG16 based feature extractor and a custom classifier
This notebook is written and executed by **Dr Raheel Siddiqi** on *11-10-2019*. The notebook presents an experiment to classify X-ray images as 'NORMAL' or 'containing manifestation of Tuberculosis (TB)' i.e. it is a binary classification problem. Transfer Learning is used to exploit the feature extractor of the *VGG16* pre-trained model. The dataset used is: [**China Set - The Shenzhen set - Chest X-ray Database**](https://www.kaggle.com/kmader/pulmonary-chest-xray-abnormalities). 10-fold cross validation is used to evaluate the model.

## TensorFlow and Keras version used

In [1]:
import tensorflow as tf
from tensorflow.python import keras

print('Tensorflow Version: ', tf.__version__)
print('Keras Version: ', keras.__version__)

Tensorflow Version:  1.14.0
Keras Version:  2.2.4-tf


## Model Setup

In [2]:
from tensorflow.python.keras.applications import VGG16
import os
import numpy as np
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import optimizers

def get_model():
    model = models.Sequential()
    conv_base=VGG16(weights='imagenet',include_top=False,input_shape=(100,100,3))
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=1e-4),metrics=['accuracy'])
    return model

In [3]:
image_height = 100
image_width = 100
batch_size = 4
no_of_epochs  = 10

## Dataset Preparation

In [4]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
%matplotlib inline

DATADIR = '/kaggle/input/pulmonary-chest-xray-abnormalities/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png'
data=[]
for img in tqdm(os.listdir(DATADIR)):
    try:
        img_array = cv2.imread(os.path.join(DATADIR,img))
        img_array = cv2.resize(img_array, (image_height, image_width))
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)        
        img_array = img_array.astype(np.float32)/255.
        if img[-5]=='0':
            data.append([img_array, 0])
        else:
            data.append([img_array, 1])
    except Exception as e:   
            pass
print(len(data))

100%|██████████| 663/663 [01:11<00:00,  9.29it/s]

662


In [5]:
import random

random.shuffle(data)
for sample in data[:10]:
    print(sample[1])

0
1
1
0
1
0
0
1
1
1


In [6]:
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, image_width, image_height, 3)
print(X.shape)

(662, 100, 100, 3)


## 10-Fold Cross Validation

In [7]:
k=10
num_validation_samples=len(X)//k
validation_scores=[]
for fold in range(k):
    validation_data=X[num_validation_samples*fold:num_validation_samples*(fold+1)]
    validation_labels=y[num_validation_samples*fold:num_validation_samples*(fold+1)]
    if fold==0:
        training_data=X[num_validation_samples*(fold+1):]
        training_labels=y[num_validation_samples*(fold+1):]    
    else:
        training_data=np.append(X[:num_validation_samples*fold], X[num_validation_samples*(fold+1):],axis=0)
        training_labels=np.append(y[:num_validation_samples*fold], y[num_validation_samples*(fold+1):],axis=0)
    model=get_model()
    model.fit(training_data,training_labels,batch_size=batch_size,epochs=no_of_epochs) # 10 epochs per model
    validation_score=model.evaluate(validation_data,validation_labels)
    validation_scores.append(validation_score[1])

58892288/58889256 [==============================] - 2s 0us/step
Epoch 1/10
596/596 [==============================] - 5s 9ms/sample - loss: 0.6546 - acc: 0.6493
Epoch 2/10
596/596 [==============================] - 3s 6ms/sample - loss: 0.5001 - acc: 0.8070
Epoch 3/10
596/596 [==============================] - 3s 6ms/sample - loss: 0.4594 - acc: 0.8104
Epoch 4/10
596/596 [==============================] - 3s 6ms/sample - loss: 0.4490 - acc: 0.8339
Epoch 5/10
596/596 [==============================] - 3s 6ms/sample - loss: 0.4171 - acc: 0.8389
Epoch 6/10
596/596 [==============================] - 3s 6ms/sample - loss: 0.3893 - acc: 0.8456
Epoch 7/10
596/596 [==============================] - 3s 6ms/sample - loss: 0.3810 - acc: 0.8557
Epoch 8/10
596/596 [==============================] - 3s 6ms/sample - loss: 0.3593 - acc: 0.8607
Epoch 9/10
596/596 [==============================] - 3s 6ms/sample - loss: 0.3514 - acc: 0.8607
Epoch 10/10
66/66 [==============================] - 0s 5ms/sa

596/596 [==============================] - 4s 6ms/sample - loss: 0.3624 - acc: 0.8607
Epoch 8/10
596/596 [==============================] - 4s 6ms/sample - loss: 0.3483 - acc: 0.8674
Epoch 9/10
596/596 [==============================] - 4s 6ms/sample - loss: 0.2965 - acc: 0.8674
Epoch 10/10
66/66 [==============================] - 0s 6ms/sample - loss: 0.5504 - acc: 0.7727
Epoch 1/10
596/596 [==============================] - 5s 8ms/sample - loss: 0.6538 - acc: 0.6342
Epoch 2/10
596/596 [==============================] - 4s 6ms/sample - loss: 0.5038 - acc: 0.7953
Epoch 3/10
596/596 [==============================] - 4s 6ms/sample - loss: 0.4822 - acc: 0.8037
Epoch 4/10
596/596 [==============================] - 4s 6ms/sample - loss: 0.4458 - acc: 0.8221
Epoch 5/10
596/596 [==============================] - 4s 6ms/sample - loss: 0.4304 - acc: 0.8272
Epoch 6/10
596/596 [==============================] - 4s 6ms/sample - loss: 0.4030 - acc: 0.8372
Epoch 7/10
596/596 [======================

## Average Validation Score

In [8]:
print('Average Validation Score: ', np.average(validation_scores))

Average Validation Score:  0.8227273
